"Projecto" : "Análisis_criptomonedas"  
"Título" : "Regresión del precio de Bitcoin"  
"Autor" : "Cristian García Díaz"  
"Fecha de creación" : "20180821"  
"Fecha de modificación" : "20180826"  
"Fuentes":  
>http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html  
>http://www.aprendemachinelearning.com/regresion-lineal-en-espanol-con-python/  
>http://davidcoallier.com/blog/linear-regression-from-r-to-python/  

>https://datatofish.com/multiple-linear-regression-python/  
>http://www.aprendemachinelearning.com/regresion-lineal-en-espanol-con-python/  
>https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9  

# Cryptocurrency Regression

### Regresión del precio del Bitcoin

## Índice
[1. Configuración del entorno y obtención de los datos](#1)  
[2. Indicadores del mercado Bitcoin](#2)  
[3. Correlación de los indicadores de Bitcoin](#3)  
[4. Obtención y análisis del volumen cambio entre *Bitcoin* y divisas](#4) 

## <a name="1"></a> 1. Configuración del entorno y obtención de los datos

   - Instalar Anaconda.  
   - Instalar las librerias, dependencias y paquetes necesarios.  
   - Crear un entorno de trabajo.
   - Estructura de carpetas para almacenar los datos.  
   - Configuración de la API Key.  
   - Función para obtención de datos desde las APIs. 

In [3]:
# Se importan las líbrerias, dependencias o paquetes necesarios
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import plotly as py
import os

# Se importa el paquete Plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
# Se configura el modo offline
py.init_notebook_mode(connected=True)

In [4]:
# API Key Quandl
quandl.ApiConfig.api_key = "gjodR_eNGkrTQq24cufg"

In [5]:
# Comprobar si no esta creada la carpeta de archivos para almacenar los datos
if not os.path.exists("cryptocurrency_indicators_files"):
    os.mkdir('cryptocurrency_indicators_files')

In [6]:
# Se define una función Quandl para cargar los datos
"""pickle --> para no descargar de nuevo los mismo datos"""
"""La función devuelve un Dataframe Pandas"""

def get_quandl_data(quandl_id):
    """Descargamos en cache los datos de Quandl"""
    """Se almacena un fichero .pkl como cache de los datos"""
    cache_path='.\cryptocurrency_indicators_files\{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path,'rb')
        df = pickle.load(f)
        print('Dataset {} cargado del cache'.format(quandl_id))
    except (OSError,IOError)as e:
        print('Descargando {} de Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cargado {} de {} en el cache'.format(quandl_id,cache_path))
    return df

In [7]:
# Se define la función para visualizar los datos
def df_scatter(df, title,seperate_y_axis=False, y_axis_label='',scale='linear',initial_hide=False):
    # Se definen la lista de los nombres de cada dataframe como una lista label_arr = ['BITSTAMP', 'COINBASE', 'ITBIT', 'KRAKEN']
    label_arr = list(df)
    # Aplicamos una función lambda para mapear cada columnas y asignar la etiqueta correspondiente
    # Se guarda como otra lista series_arr
    series_arr = list(map(lambda col:df[col],label_arr))
    
    # Se definen los parametros de la salida gráfica
    layout = go.Layout(
        title = title,
        legend = dict(orientation='h'),
        xaxis = dict(type='date'),
        yaxis = dict(
            title = y_axis_label, 
            showticklabels = not seperate_y_axis,
            type = scale
        )
    )
    
    # Se define la configuración del eje y
    y_axis_config = dict(
        overlaying = 'y',
        showticklabels = False,
        type = scale
    )
    
    # Se define la visibilidad
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Se define la forma para cada serie de datos
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
        x = series.index,
        y = series,
        name = label_arr[index],
        visible = visibility
        )
        
        #Añadir un eje separado para cada serie
        if seperate_y_axis:
            trace['yaxis'] = 'y{format}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config
        trace_arr.append(trace)
    
    fig = go.Figure(data = trace_arr, layout = layout)
    py.iplot(fig)

## <a name="2"></a> 2. Indicadores del mercado Bitcoin
   - [2.1 Precio de Bitcoin](#2.1)
   - [2.2 Número total de bictoins](#2.2)
   - [2.3 Valor del mercado](#2.3) 
   - [2.4 Direcciones de bitcoin](#2.4) 
   - [2.5 Volumen de cambio de bitcoin a dóladores](#2.5) 
   - [2.6 Número de transacciones de bitcoin](#2.6) 
   - [2.7 Número de transacciones acumuladas de bitcoin](#2.7) 
   - [2.8 Hash rate de bitcoin](#2.8) 
   - [2.9 Dificultad de bitcoin](#2.9) 
   - [2.10 Recompensa de los mineros de bitcoin](#2.10) 
   
   "Fuentes":  
>https://www.quandl.com/data/BCHAIN-Blockchain

### <a name="2.1"></a> 2.1 Precio de Bitcoin

In [8]:
# Obtención de los datos del precio
price_btc = get_quandl_data("BCHAIN/MKPRU")

Dataset BCHAIN/MKPRU cargado del cache


### <a name="2.2"></a> 2.2 Número total de bictoins

In [9]:
# Se obtienen los datos históricos del número total de Bitcoins
total_number_btc = get_quandl_data("BCHAIN/TOTBC")

Dataset BCHAIN/TOTBC cargado del cache


### <a name="2.3"></a> 2.3 Valor del mercado

In [10]:
# Se obtienen los datos históricos de la capitalización del mercado del Bitcoin en USD. El valor de mercado del Bitcoin
market_capitalization_btc = get_quandl_data("BCHAIN/MKTCP")

Dataset BCHAIN/MKTCP cargado del cache


### <a name="2.4"></a> 2.4 Número de direcciones bitcoin

In [11]:
# Se obtienen los datos históricos del número de direcciones Bitcoin usadas por dia
address_btc = get_quandl_data("BCHAIN/NADDU")

Dataset BCHAIN/NADDU cargado del cache


### <a name="2.5"></a> 2.5 Volumen de cambio de USD/BTC

In [12]:
# Se obtienen los datos históricos del volumen de cambio USD/BTC
exchange_trade_btc = get_quandl_data("BCHAIN/TRVOU")

Dataset BCHAIN/TRVOU cargado del cache


### <a name="2.6"></a> 2.6 Número de transacciones Bitcoin

In [13]:
# Se obtienen los datos históricos de las transacciones de BTC
transactions_btc = get_quandl_data("BCHAIN/NTRAN")

Dataset BCHAIN/NTRAN cargado del cache


### <a name="2.8"></a> 2.8 Hash rate de bitcoin 

In [14]:
# Se obtienen los datos históricos de Hash rate de bitcoin
# Es el número estimado de hash rate de Bitcoun y se miden en TeraHashes por segundo TH/s.
# 1 TH/s = 10^12 = 1.000.000.000.000 hash/s = 1 billón de hashes por segundo.
hash_rate_btc = get_quandl_data("BCHAIN/HRATE")

Dataset BCHAIN/HRATE cargado del cache


### <a name="2.9"></a> 2.9 Dificultad de bitcoin

In [15]:
# Se obtienen la dificultad de Bitcoin. Es una medida de dificultad propia.
# cada 210.000 bloques se recalcula la dificultad para crear bloques en la cadena de bloques cada 10 minutos de media.
difficulty_btc = get_quandl_data("BCHAIN/DIFF")

Dataset BCHAIN/DIFF cargado del cache


### <a name="2.10"></a> 2.10 Recompensa de los mineros de bitcoin 

In [16]:
# Se obtienen los datos históricos de la recompensa de los mineros
miners_revenue_btc = get_quandl_data("BCHAIN/MIREV")

Dataset BCHAIN/MIREV cargado del cache


## <a name="3"></a>3. Correlación de los indicadores de Bitcoin

In [17]:
# Se preparan los datos para unirlos en un único Dataframe para poder aplicar la correlación.
mesures_name= ["price_btc",
        "total_number_btc",
        "market_capitalization_btc",
        "address_btc",
        "exchange_trade_btc",
        "transactions_btc",
        "hash_rate_btc",
        "difficulty_btc",
        "miners_revenue_btc"]

mesures_data= [price_btc,
        total_number_btc,
        market_capitalization_btc,
        address_btc,
        exchange_trade_btc,
        transactions_btc,
        hash_rate_btc,
        difficulty_btc,
        miners_revenue_btc]

In [18]:
# Preparación de un Dataframe con todos los indicadores e igualar los días para construir para todos los indicadores
fecha=pd.Timestamp(2018, 8, 26)

for i in range(len(mesures_name)):
    if(mesures_data[i].index.max()== fecha):
        mesures_data[i] = mesures_data[i].drop(mesures_data[i].index[len(mesures_data[i])-1])
    print( i, mesures_name[i], mesures_data[i].index.max()-mesures_data[i].index.min(), mesures_data[i].index.max())

0 price_btc 3521 days 00:00:00 2018-08-25 00:00:00
1 total_number_btc 3521 days 00:00:00 2018-08-25 00:00:00
2 market_capitalization_btc 3521 days 00:00:00 2018-08-25 00:00:00
3 address_btc 3521 days 00:00:00 2018-08-25 00:00:00
4 exchange_trade_btc 3521 days 00:00:00 2018-08-25 00:00:00
5 transactions_btc 3521 days 00:00:00 2018-08-25 00:00:00
6 hash_rate_btc 3520 days 00:00:00 2018-08-25 00:00:00
7 difficulty_btc 3521 days 00:00:00 2018-08-25 00:00:00
8 miners_revenue_btc 3521 days 00:00:00 2018-08-25 00:00:00


In [19]:
# Se realiza una unión de los varios Dataframe en uno
mesures_bitcoin = pd.concat(
    [mesures_data[0],
     mesures_data[1],
     mesures_data[2],
     mesures_data[3],
     mesures_data[4],
     mesures_data[5],
     mesures_data[6],
     mesures_data[7],
     mesures_data[8]],axis=1
)

In [20]:
# Renombrar las columnas
for i in range(len(mesures_name)):
    mesures_bitcoin.columns.values[i] = mesures_name[i]

In [21]:
mesures_bitcoin.drop(['market_capitalization_btc'],axis=1)

,price_btc,total_number_btc,address_btc,exchange_trade_btc,transactions_btc,hash_rate_btc,difficulty_btc,miners_revenue_btc
Date,,,,,,,,
2009-01-03,0.000000,50.0,1.0,0.000000e+00,1.0,NaN,1.000000e+00,0.000000e+00
2009-01-04,0.000000,50.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
2009-01-05,0.000000,50.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
2009-01-06,0.000000,50.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
2009-01-07,0.000000,50.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
2009-01-08,0.000000,50.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
2009-01-09,0.000000,750.0,14.0,0.000000e+00,14.0,6.959438e-07,1.000000e+00,0.000000e+00
2009-01-10,0.000000,2300.0,31.0,0.000000e+00,31.0,1.541018e-06,1.000000e+00,0.000000e+00
2009-01-11,0.000000,7600.0,106.0,0.000000e+00,106.0,5.269289e-06,1.000000e+00,0.000000e+00


In [22]:
mesures_bitcoin.tail()

,price_btc,total_number_btc,market_capitalization_btc,address_btc,exchange_trade_btc,transactions_btc,hash_rate_btc,difficulty_btc,miners_revenue_btc
Date,,,,,,,,,
2018-08-21,6434.559167,17221487.5,1.108127e+11,459855.0,2.784076e+08,214983.0,4.764220e+07,6.389317e+12,1.225187e+07
2018-08-22,6401.246154,17223337.5,1.102508e+11,449915.0,4.942757e+08,217777.0,4.700697e+07,6.389317e+12,1.200075e+07
2018-08-23,6575.229167,17225337.5,1.132605e+11,484462.0,5.786947e+08,230333.0,5.081835e+07,6.389317e+12,1.331007e+07
2018-08-24,6434.881667,17227200.0,1.108550e+11,444345.0,5.214981e+08,217352.0,4.732459e+07,6.389317e+12,1.210921e+07
2018-08-25,6543.645714,17229025.0,1.127406e+11,487990.0,3.094877e+08,219848.0,4.882418e+07,6.505039e+12,1.206749e+07


In [23]:
# Correlación de las variables
mesures_correlacion = mesures_bitcoin.corr()

In [24]:
# Correlación entre las variables
mesures_correlacion

,price_btc,total_number_btc,market_capitalization_btc,address_btc,exchange_trade_btc,transactions_btc,hash_rate_btc,difficulty_btc,miners_revenue_btc
price_btc,1.000000,0.465134,0.999782,0.677478,0.853488,0.582283,0.779903,0.775728,0.980981
total_number_btc,0.465134,1.000000,0.458457,0.808610,0.342171,0.818690,0.420175,0.421509,0.457854
market_capitalization_btc,0.999782,0.458457,1.000000,0.672858,0.852266,0.577893,0.786708,0.782611,0.978755
address_btc,0.677478,0.808610,0.672858,1.000000,0.568590,0.983264,0.514078,0.511148,0.674568
exchange_trade_btc,0.853488,0.342171,0.852266,0.568590,1.000000,0.482211,0.560481,0.554420,0.865675
transactions_btc,0.582283,0.818690,0.577893,0.983264,0.482211,1.000000,0.449453,0.446921,0.577882
hash_rate_btc,0.779903,0.420175,0.786708,0.514078,0.560481,0.449453,1.000000,0.994816,0.698855
difficulty_btc,0.775728,0.421509,0.782611,0.511148,0.554420,0.446921,0.994816,1.000000,0.685947
miners_revenue_btc,0.980981,0.457854,0.978755,0.674568,0.865675,0.577882,0.698855,0.685947,1.000000


In [25]:
# Correlación entre las variables
mesures_correlacion['price_btc'].sort_values(ascending=False)

price_btc                    1.000000
market_capitalization_btc    0.999782
miners_revenue_btc           0.980981
exchange_trade_btc           0.853488
hash_rate_btc                0.779903
difficulty_btc               0.775728
address_btc                  0.677478
transactions_btc             0.582283
total_number_btc             0.465134
Name: price_btc, dtype: float64

## <a name="4"></a>4. Regresión Lineal

In [26]:
# Crear los datos de entreno, test y evaluación
# Entreno
train_from_date = '2016-11-22'
train_end_date =  '2018-06-22'
# Test
test_from_date = '2018-06-23'
test_end_date = '2018-08-16'
# Evaluación
evaluation_from_date = '2018-08-17'
evaluation_end_date = '2018-08-22'

df_train = mesures_bitcoin.loc[train_from_date:train_end_date]
df_test = mesures_bitcoin.loc[test_from_date:test_end_date]
df_evaluation = mesures_bitcoin.loc[evaluation_from_date:evaluation_end_date]


print(df_train.size," días de entreno\n",df_test.size," días de test\n",df_evaluation.size," días de evaluación\n")

train_days = mesures_bitcoin.loc[train_from_date:train_end_date].count()
test_days = mesures_bitcoin.loc[test_from_date:test_end_date].count()
evalutacion_days = mesures_bitcoin.loc[evaluation_from_date:evaluation_end_date].count()
print(train_days, " desde ",train_from_date," hasta ",train_end_date )
print(test_days, " desde ",test_from_date," hasta ",test_end_date )
print(evalutacion_days, " desde ",evaluation_from_date," hasta ",evaluation_end_date )

5202  días de entreno
 495  días de test
 54  días de evaluación

price_btc                    578
total_number_btc             578
market_capitalization_btc    578
address_btc                  578
exchange_trade_btc           578
transactions_btc             578
hash_rate_btc                578
difficulty_btc               578
miners_revenue_btc           578
dtype: int64  desde  2016-11-22  hasta  2018-06-22
price_btc                    55
total_number_btc             55
market_capitalization_btc    55
address_btc                  55
exchange_trade_btc           55
transactions_btc             55
hash_rate_btc                55
difficulty_btc               55
miners_revenue_btc           55
dtype: int64  desde  2018-06-23  hasta  2018-08-16
price_btc                    6
total_number_btc             6
market_capitalization_btc    6
address_btc                  6
exchange_trade_btc           6
transactions_btc             6
hash_rate_btc                6
difficulty_btc               6

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.api as sm

### Regresión linear
#### El estimador utilizado es OLS

In [28]:
# Variable independientes
X = mesures_bitcoin.loc["2016-01-01":'2018-06-22',"total_number_btc":]
# Variables predictoras
y = mesures_bitcoin.loc["2016-01-01":'2018-06-22',"price_btc"]

In [29]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Creamos el objeto mesures_bitcoinde Regresión Linear
lm = linear_model.LinearRegression()

# Entrenamos nuestro modelo
model = lm.fit(X,y)

# Calcular la predicción
predictions = lm.predict(X)

print(predictions[:5])
print(predictions[-5:])

[409.62148297 410.33031868 424.91824155 413.19703418 421.47469707]
[6430.30487311 6687.03857803 6707.64679591 6600.42530862 6700.20256737]


In [30]:
X

,total_number_btc,market_capitalization_btc,address_btc,exchange_trade_btc,transactions_btc,hash_rate_btc,difficulty_btc,miners_revenue_btc
Date,,,,,,,,
2016-01-01,15028600.0,6.452379e+09,310331.0,2.854093e+07,164132.0,7.436044e+05,1.038803e+11,1.557216e+06
2016-01-02,15031975.0,6.498774e+09,232199.0,2.040696e+07,123623.0,6.971292e+05,1.038803e+11,1.467328e+06
2016-01-03,15035400.0,6.524461e+09,334703.0,1.506949e+07,142904.0,7.074570e+05,1.038803e+11,1.498829e+06
2016-01-04,15039125.0,6.438701e+09,295177.0,2.332215e+07,141064.0,7.694240e+05,1.038803e+11,1.604631e+06
2016-01-05,15043500.0,6.513836e+09,363742.0,2.194946e+07,170176.0,9.036860e+05,1.038803e+11,1.906932e+06
2016-01-06,15047575.0,6.499048e+09,365051.0,2.048775e+07,187012.0,8.417189e+05,1.038803e+11,1.774245e+06
2016-01-07,15050975.0,6.483207e+09,338929.0,1.915684e+07,177478.0,7.022931e+05,1.038803e+11,1.477472e+06
2016-01-08,15054575.0,6.830411e+09,326219.0,5.847808e+07,177873.0,7.436044e+05,1.038803e+11,1.648782e+06
2016-01-09,15058950.0,6.731953e+09,366508.0,4.119154e+07,190001.0,9.036860e+05,1.038803e+11,1.971446e+06


In [31]:
y[1]

432.33

In [32]:
print("Los coeficientes son: \n")
for i in range(len(mesures_name)-1):
    print( lm.coef_[i] ,"  ",mesures_name[i])

Los coeficientes son: 

2.9613743447402266e-05    price_btc
5.9599257883422716e-08    total_number_btc
0.00018292774944211204    market_capitalization_btc
3.0534857587459157e-09    address_btc
-0.0002989148074063619    exchange_trade_btc
-4.6016170935714627e-07    transactions_btc
-4.792408171306701e-11    hash_rate_btc
-3.451593272720719e-07    difficulty_btc


In [33]:
# MSE
print("MSE es :\n",mean_squared_error(y, predictions))

MSE es :
 236.58691335514652


In [34]:
from math import sqrt
# RMSE
print("RMSE es :\n",sqrt(mean_squared_error(y, predictions)))

RMSE es :
 15.381382036577419


In [35]:
from sklearn.metrics import mean_absolute_error
# MAE
print("MAE es :\n",mean_absolute_error(y, predictions))

MAE es :
 11.014176202235772


In [36]:
print("La varianza es:\n",lm.score(X,y))


La varianza es:
 0.9999864906730572


In [37]:
print("La varianza es:\n",r2_score(y, predictions))

La varianza es:
 0.9999864906730572


In [38]:
lm.intercept_

-421.9244088084256

In [39]:
# Se añade la constante
X = sm.add_constant(X)
# Se crea el modelo de regresión
model = sm.OLS(y, X).fit()
# Se crea el modelo de regresión
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:              price_btc   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.281e+06
Date:                Sun, 09 Sep 2018   Prob (F-statistic):               0.00
Time:                        11:36:37   Log-Likelihood:                -3753.5
No. Observations:                 904   AIC:                             7525.
Df Residuals:                     895   BIC:                             7568.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [40]:
# Crear Dataframe de las prediciones
pred=pd.DataFrame(predictions)

In [41]:
# Renombrar la columna de Dataframe
pred =pred.rename(columns={0:'prediction_btc'})

In [42]:
# Crear Dataframe de las observaciones
obs=pd.DataFrame(y)

In [43]:
# Renombrar la columna de Dataframe
obs = obs.rename(columns={'price_btc':'observation_btc'})

In [44]:
# Dataframe resultado
result = obs.join(pred,lsuffix='_obs', rsuffix='_pred')

In [45]:
# Gráfica del dataframe de resultado
df_scatter(result,'Obervaciones vs predicciones')

In [46]:
y = mesures_bitcoin.loc["2018-06-23":'2018-08-16',"price_btc"]

In [47]:
X = mesures_bitcoin.loc["2018-06-23":"2018-08-16","total_number_btc":]

In [48]:
# Calcular la predicción
predictions = lm.predict(X)

In [49]:
predictions

array([6702.74323351, 6008.00232696, 6143.1800252 , 6119.2392871 ,
       6208.5235807 , 6103.66504549, 6103.34568187, 5920.69992916,
       5913.61937846, 6321.82230795, 6364.99699089, 6601.71409743,
       6586.80768772, 6662.66149445, 6491.42147706, 6584.42404174,
       6578.98308328, 6691.56735083, 6673.2262642 , 6370.70046017,
       6355.38934772, 6130.6946347 , 6169.43321742, 6234.26463334,
       6332.88574201, 6651.3629446 , 7360.42530427, 7324.75288606,
       7514.02382169, 7334.23252131, 7416.55620166, 7417.09133718,
       7724.02377656, 8466.95190487, 8225.42873659, 7965.52518079,
       8171.82448479, 8204.48825703, 8245.62275566, 8155.16245599,
       7683.79566706, 7572.22969588, 7543.59058205, 7414.89318852,
       6975.03063691, 7006.82730611, 6848.41287259, 6411.15806228,
       6356.17366796, 6358.66742221, 6129.46283198, 6244.63699277,
       6283.79222734, 5986.59589768, 6307.79216513])

In [50]:
pred=pd.DataFrame(predictions)

In [51]:
# Renombrar la columna de Dataframe
pred =pred.rename(columns={0:'prediction_btc'})

In [52]:
obs=pd.DataFrame(y)

In [53]:
obs = obs.reset_index()

In [54]:
obs= obs.drop(columns=['Date'],axis=1)

In [55]:
result = obs.join(pred,lsuffix='_obs', rsuffix='_pred')

In [56]:
TestDate =  mesures_bitcoin.loc["2018-06-23":"2018-08-16",'total_number_btc':'total_number_btc'].index

In [57]:
# Visualización de 
trace1 = go.Scatter(

    x = TestDate,
    y = result['prediction_btc'],
    mode = 'lines',
    name = 'Predicted',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = TestDate,
    y = result['price_btc'],
    mode = 'lines',
    name = 'Observated',
    line = dict(color=('rgb(66, 146, 244)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

In [59]:
from sklearn.metrics import mean_absolute_error
# MAE
print("MAE es :\n",mean_absolute_error(result['price_btc'], result['prediction_btc']))

MAE es :
 96.9317701667584


In [60]:
y = mesures_bitcoin.loc['2018-08-17':'2018-08-22',"price_btc"]
X = mesures_bitcoin.loc['2018-08-17':'2018-08-22',"total_number_btc":]
# Calcular la predicción
predictions = lm.predict(X)
pred=pd.DataFrame(predictions)
# Renombrar la columna de Dataframe
pred =pred.rename(columns={0:'prediction_btc'})
obs=pd.DataFrame(y)
obs = obs.reset_index()
obs= obs.drop(columns=['Date'],axis=1)
result = obs.join(pred,lsuffix='_obs', rsuffix='_pred')
TestDate =  mesures_bitcoin.loc['2018-08-17':'2018-08-22','total_number_btc':'total_number_btc'].index

In [61]:
# Visualización de 
trace1 = go.Scatter(

    x = TestDate,
    y = result['prediction_btc'],
    mode = 'lines',
    name = 'Predicted',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = TestDate,
    y = result['price_btc'],
    mode = 'lines',
    name = 'Observated',
    line = dict(color=('rgb(66, 146, 244)'), width=2)
)

data = [trace1, trace2]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

In [62]:
from sklearn.metrics import mean_absolute_error
# MAE
print("MAE es :\n",mean_absolute_error(result['price_btc'], result['prediction_btc']))

MAE es :
 55.49789451740526
